Don't forget to set the args marked with ########. With current params first block downloads all 60gb+ of train images (not randomly if _max is set larger than dataset)!
Make sure images/ is empty and the resize outdir does not exist when running from the first block
Viktor

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import io
import os
import sys
import json
import urllib3
import multiprocessing
import numpy as np
import random

from PIL import Image
from tqdm import tqdm
from urllib3.util import Retry

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#Chooses randomly if _max (the number of images to get) is smaller than the dataset

#Fixing the args
#example args
#downloader.py test.json test
fix_args=['whatever','train.json','alltrain']   ################################

def download_image(fnames_and_urls):
    """
    download image and save its with 90% quality as JPG format
    skip image downloading if image already exists at given path
    :param fnames_and_urls: tuple containing absolute path and url of image
    """
    fname, url = fnames_and_urls
    if not os.path.exists(fname):
        http = urllib3.PoolManager(retries=Retry(connect=3, read=2, redirect=3))
        response = http.request("GET", url)
        image = Image.open(io.BytesIO(response.data))
        image_rgb = image.convert("RGB")
        image_rgb.save(fname, format='JPEG', quality=90)


def parse_dataset(_dataset, _outdir, _max=2000000):     ################################
    """
    parse the dataset to create a list of tuple containing absolute path and url of image
    :param _dataset: dataset to parse
    :param _outdir: output directory where data will be saved
    :param _max: maximum images to download (change to download all dataset)
    :return: list of tuple containing absolute path and url of image
    """
    _fnames_urls = []
    with open(dataset, 'r') as f:
        data = json.load(f)
        for image in data["images"]:
            url = image["url"]
            fname = os.path.join(outdir, "{}.jpg".format(image["imageId"]))
            _fnames_urls.append((fname, url))
            
        #randomize
        random_choice = []
        if _max < len(_fnames_urls):
            random_choice = np.random.choice(len(_fnames_urls), _max, replace=False).tolist()
            _fnames_urls = [_fnames_urls[i] for i in random_choice]
        
    return _fnames_urls, random_choice

if __name__ == '__main__':
    if len(fix_args) != 3:
        print("error: not enough arguments")
        #sys.exit(0)

    # get args and create output directory
    dataset, outdir = fix_args[1:]
    if not os.path.exists(outdir):
        os.makedirs(outdir)
        print('made output dir called '+outdir)

    # parse json dataset file
    fnames_urls, random_choice = parse_dataset(dataset, outdir)
    print(len(random_choice))
    #ranc = open('random_choice.txt', 'w')
    #for item in random_choice:
        #ranc.write("%s\n" % item)
    with open('random_choice.csv', 'w') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(random_choice)
    
    # download data
    pool = multiprocessing.Pool(processes=12)
    with tqdm(total=len(fnames_urls)) as progress_bar:
        for _ in pool.imap_unordered(download_image, fnames_urls):
            progress_bar.update(1)

    #sys.exit(1)
    print('done')

0


100%|██████████| 1014544/1014544 [01:26<00:00, 11678.34it/s]

done


In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import os, sys
print("TensorFlow version:", tf.__version__)

#rescale and pad with white (3 minutes for 10k 256x256)
# combined https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/ + https://gist.github.com/ihercowitz/642650/f01986c0b1ebd04be588b196eb3ffefe9853e113

def resizeImage(infile, output_dir="scaled", desired_size = 299):
     outfile = os.path.splitext(infile)[0]
     extension = os.path.splitext(infile)[1]

     #if (cmp(extension, ".jpg")):
     if extension != ".jpg":
        return

     if infile != outfile:
        im = Image.open(dir+infile)
        old_size = im.size  # old_size[0] is in (width, height) format
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        im = im.resize(new_size, Image.ANTIALIAS)
        new_im = Image.new("RGB", (desired_size, desired_size),color=(255,255,255))
        new_im.paste(im, ((desired_size-new_size[0])//2,(desired_size-new_size[1])//2))
        new_im.save(output_dir+outfile+extension,"JPEG")
        #except IOError:
            #print("cannot reduce image for ", infile)


if __name__=="__main__":
    dir = 'alltrain/'    ################################
    desired_size = 299
    output_dir = "images/alltrain299/"    ################################
    #dir = os.getcwd()
    

    if not os.path.exists(os.path.join(dir,output_dir)):    #######
        os.mkdir(output_dir)

    for file in os.listdir(dir):
        resizeImage(file,output_dir,desired_size)

TensorFlow version: 1.7.0


In [ ]:
#check pipelog.txt to see whether retrain.py is actually working!

In [ ]:
!nohup python retrain.py --bottleneck_dir=bottlenecks --how_many_training_steps 500 --model_dir=model_dir --output_graph=retrained_graph.pb --output_labels=retrained_labels.txt --summaries_dir=retrain_logs --image_dir=images > pipelog.txt 2>&1

!nohup python3 retrain.py --bottleneck_dir=bottlenecks --how_many_training_steps 500 --model_dir=model_dir --output_graph=retrained_graph.pb --output_labels=retrained_labels.txt --summaries_dir=retrain_logs --image_dir=images > pipelog.txt 2>&1

In [ ]:
import csv
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageOps
import os, sys
print("TensorFlow version:", tf.__version__)

#1 Predict probabilities for labels, save .csv

#Runtime 109s for 100 (155kB, with .4 float precision, on Intel i7), 1445s for 1000 imgs (1.6MB)

#params
input_path = 'test/'
first = 1
last = 39706     ################################
image_numbers = range(first, last+1)
save_path = 'predictions/'
save_as = 'ffall0.csv'     ################################

start = time.time()

#modified from https://github.com/BartyzalRadek/Multi-label-Inception-net
# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("labels.txt")]
top_k = np.asarray([int(lbl) for lbl in label_lines])

all_predictions = np.empty([len(image_numbers),len(label_lines)+1])

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')    
    
    #iterate over images
    counter = 0
    for i, image_num in enumerate(image_numbers):
        image_path = input_path+str(image_num)+'.jpg'
        #print(image_path)
        # Read in the image_data
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()



        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        all_predictions[i] = np.insert(predictions, 0, image_num)

        if (counter == 5000):
            print(image_num)
            np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')
            counter = 0

        counter += 1

np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')

#header


end = time.time()
print(end - start)
print('Saved to '+save_path+save_as)

In [ ]:
input_path = 'validation/'
first = 1
last = 9897     ################################
image_numbers = range(first, last+1)
save_path = 'predictions/'
save_as = 'ffall0val.csv'     ################################

start = time.time()

#modified from https://github.com/BartyzalRadek/Multi-label-Inception-net
# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("labels.txt")]
top_k = np.asarray([int(lbl) for lbl in label_lines])

all_predictions = np.empty([len(image_numbers),len(label_lines)+1])

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')    
    
    #iterate over images
    counter = 0
    for i, image_num in enumerate(image_numbers):
        image_path = input_path+str(image_num)+'.jpg'
        #print(image_path)
        # Read in the image_data
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()



        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        all_predictions[i] = np.insert(predictions, 0, image_num)

        if (counter == 5000):
            print(image_num)
            np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')
            counter = 0

        counter += 1

np.savetxt(save_path+save_as, all_predictions, delimiter=",", fmt='%1.4f')

#header


end = time.time()
print(end - start)
print('Saved to '+save_path+save_as)